## 2. Data Cleansing and Data Loading 

After we found additional data, we made 'Fulldata_air(daily).csv' by combining all data.

Our data has some missing values (especiallly about fish price), however it is quite clean, barely had to check outliers. 
We just check whether the value of AQI index is more than 500, or the value of fish price is more than 4000. <p>
Then we use linear interpolation by index because it is a time-series data. 

In [1]:
import numpy as np
import pandas as pd
from itertools import product
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor


# load data
sample = pd.read_csv("./Fulldata_air(daily).csv", index_col=0)
#sample.fillna(value=-1, inplace=True)

S_AQIset=['PM10_AQI_0', 'PM2_5_AQI_0','PM10_AQI_1', 'PM2_5_AQI_1','PM10_AQI_2', 'PM2_5_AQI_2','PM10_AQI_3', 'PM2_5_AQI_3','PM10_AQI_4', 'PM2_5_AQI_4','PM10_AQI_5', 'PM2_5_AQI_5','PM10_AQI_6', 'PM2_5_AQI_6','PM10_AQI_7', 'PM2_5_AQI_7','PM10_AQI_8', 'PM2_5_AQI_8','PM10_AQI_9', 'PM2_5_AQI_9','PM10_AQI_10', 'PM2_5_AQI_10','PM10_AQI_11', 'PM2_5_AQI_11','PM10_AQI_12', 'PM2_5_AQI_12','PM10_AQI_13', 'PM2_5_AQI_13','PM10_AQI_14', 'PM2_5_AQI_14','PM10_AQI_15', 'PM2_5_AQI_15','PM10_AQI_16', 'PM2_5_AQI_16','PM10_AQI_17', 'PM2_5_AQI_17','PM10_AQI_18', 'PM2_5_AQI_18','PM10_AQI_19', 'PM2_5_AQI_19','PM10_AQI_20', 'PM2_5_AQI_20','PM10_AQI_21', 'PM2_5_AQI_21','PM10_AQI_22', 'PM2_5_AQI_22','PM10_AQI_23', 'PM2_5_AQI_23','PM10_AQI_24', 'PM2_5_AQI_24','PM10_AQI_25', 'PM2_5_AQI_25']
B_AQIset=['PM10_DAY_AVE','PM2_5_DAY_AVE']
# check outliers -> np.nan
# fill missing data: -1 -> np.nan
for index, feature in product(sample.index, sample.columns):
    data = sample.loc[index, feature]
    
    #Outlier : Seoul AQI > 500 or Beijing AQI > 500 or FISH_AVE_PR2ICE_DAY > 4000
    if feature in S_AQIset:
        if data > 500 :  
            sample.loc[index, feature] = np.nan
            feature1=feature.replace("AQI", "CONC");
            sample.loc[index, feature1] = np.nan
    if feature in B_AQIset: 
        if data > 500 :  
            sample.loc[index, feature] = np.nan
    if feature=='FISH_AVE_PRICE_DAY':
        if data > 4000 : 
            sample.loc[index, 'FISH_TOT_PRICE_DAY'] = np.nan
            sample.loc[index, 'FISH_TOT_NUM_DAY'] = np.nan
            sample.loc[index, 'FISH_AVE_PRICE_DAY'] = np.nan
            
    # missing data 
    if data == -1:
        sample.loc[index, feature] = np.nan
        
# np.nan-> linear interpolation by index
sample.interpolate(method='linear', axis=0, inplace=True, limit_direction="forward")
sample = sample.dropna()

In [2]:
sample.describe()

,PM10_CONC_0,PM2_5_CONC_0,O3_0,NO2_0,CO_0,SO2_0,PM10_AQI_0,PM2_5_AQI_0,PM10_CONC_1,PM2_5_CONC_1,...,PM2_5_AQI_25,PM10_DAY_AVE,PM2_5_DAY_AVE,O3_DAY_AVE,NO2_DAY_AVE,CO_DAY_AVE,SO2_DAY_AVE,FISH_TOT_PRICE_DAY,FISH_TOT_NUM_DAY,FISH_AVE_PRICE_DAY
count,1581.000000,1581.000000,1581.000000,1581.000000,1581.000000,1581.000000,1581.000000,1581.000000,1581.000000,1581.000000,...,1581.000000,1581.000000,1581.000000,1581.000000,1581.000000,1581.000000,1581.000000,1581.000000,1581.000000,1581.000000
mean,46.130482,24.825111,0.023446,0.031561,0.525079,0.005014,39.334783,76.039716,45.064542,25.361612,...,73.156981,74.978178,135.157179,44.285394,24.515444,10.517420,6.976940,184040.196078,194.857685,1026.175894
std,28.026675,13.311916,0.012161,0.012034,0.175032,0.002715,16.764644,28.435982,30.098863,14.240605,...,30.902252,51.170444,72.931292,33.771138,12.419908,9.240141,9.050478,107442.156515,113.590850,467.821840
min,4.750000,3.541667,-0.029708,-0.024792,0.208333,-0.038042,4.666667,15.166667,4.208333,2.125000,...,1.791667,5.000000,18.000000,1.000000,2.000000,1.000000,0.916667,5000.000000,16.000000,166.666667
25%,29.833333,15.750000,0.014583,0.022625,0.400000,0.004167,27.541667,57.625000,28.458333,15.375000,...,52.458333,42.000000,81.000000,21.000000,16.000000,5.000000,2.000000,105000.000000,117.000000,725.008210
50%,41.125000,22.166667,0.022583,0.029917,0.483333,0.004833,37.750000,72.125000,40.458333,22.958333,...,70.958333,65.000000,129.000000,33.000000,22.000000,8.000000,4.000000,164200.000000,178.000000,912.666667
75%,56.833333,31.041667,0.031542,0.039250,0.608333,0.005750,49.916667,91.208333,55.375000,32.291667,...,92.000000,92.000000,170.000000,61.000000,31.000000,13.000000,8.000000,247350.000000,248.000000,1204.810766
max,596.791667,99.250000,0.072208,0.080583,1.300000,0.013500,122.500000,173.666667,743.166667,92.916667,...,172.958333,472.000000,488.000000,175.000000,90.000000,78.000000,71.000000,667800.000000,847.000000,3763.541667


## 3. Algorithm Implementation 

We use TPOT with 3 different configurations, the basic TPOT regressor, TPOT regressor_light, and TPOT regressor_sparse<p>
We will explain about these algorithms in detail in our report, and you can check how they are implemented by the below part. 

## 4. Evaluate Model

Our models predict the next day's average PM10_AQI and the average of PM10_AQI after 3 days in seoul. <p>
We split the data into training and test set. 80% of data was used for training.<p>
We don't have to make validation set, because tpot algorithm contains the cross validation process. 
    (default : 5-fold CV)

In [3]:
# target value
target = "PM10_AQI_0"

# targets(=tomorrow's AQI), targets3 (=3 days after AQI)
targets = sample[target].copy()
targets = targets.shift(-1)
targets.dropna(inplace=True) 
targets3 = targets.shift(-2)
targets3.dropna(inplace=True) 

# drop the last row of the features
features = sample.copy()
features = features.iloc[:-1, :].copy()
features3 = features.iloc[:-2, :].copy()

targets = targets.to_numpy()
features = features.to_numpy()
targets3 = targets3.to_numpy()
features3 = features3.to_numpy()

# split into train and test set
X_train, X_test, y_train, y_test = train_test_split(features.astype(np.float64), targets.astype(np.float64), train_size=0.8, test_size=0.2, shuffle=False)
X_train3, X_test3, y_train3, y_test3 = train_test_split(features3.astype(np.float64), targets3.astype(np.float64), train_size=0.8, test_size=0.2, shuffle=False)

In [47]:
# Model Accuracy Standard 

from sklearn.metrics import mean_absolute_error

y_today = sample[target].copy()
y_tomorrow = y_today.shift(-1)
y_3days = y_today.shift(-3)

y_today= y_today.iloc[:-1]
y_tomorrow.dropna(inplace=True) 
mae = mean_absolute_error (y_today, y_tomorrow)
print("Average difference between today AQI and tomorrow AQI: ", mae)

y_today= y_today.iloc[:-2]
y_3days.dropna(inplace=True)
mae = mean_absolute_error (y_today, y_3days)
print("Average difference between today AQI and 3 days after AQI : ", mae)

Average difference between today AQI and tomorrow AQI:  10.626265822603798
Average difference between today AQI and 3 days after AQI :  15.456062526276298


### 4-1. Tpot Regressor 

#### model for tomorrow 

In [15]:
tpot = TPOTRegressor(generations=5, population_size=50, verbosity=2, scoring='neg_mean_absolute_error', random_state=42)
tpot.fit(X_train, y_train)
tpot.export('tpot_seoul_PM10_AQI.py')
print("neg MAE of TPOT Regressor : ",tpot.score(X_test, y_test))


Generation 1 - Current best internal CV score: -9.555932888144216

Generation 2 - Current best internal CV score: -9.503105434894564

Generation 3 - Current best internal CV score: -9.42252111141496

Generation 4 - Current best internal CV score: -9.415715981058668

Generation 5 - Current best internal CV score: -9.415715981058668

Best pipeline: RandomForestRegressor(ElasticNetCV(MaxAbsScaler(input_matrix), l1_ratio=0.8, tol=0.01), bootstrap=True, max_features=0.4, min_samples_leaf=16, min_samples_split=14, n_estimators=100)
neg MAE of TPOT Regressor :  -8.374853723205113


->>> MAE is decreased to 8.37 from 10.63

* Important Features for best pipeline (RandomForestRegressor)

In [25]:
BestPipe = tpot.fitted_pipeline_.steps[-1][1]
BestPipe.fit(X_train, y_train)


# Access it's features
feature_list = pd.concat([pd.Series(sample.columns), pd.Series(BestPipe.feature_importances_)], axis=1)
feature_list.columns = ['features_name', 'importance']
feature_list.sort_values("importance", ascending =False)[:50]

,features_name,importance
104,PM10_CONC_13,0.060590
80,PM10_CONC_10,0.059476
6,PM10_AQI_0,0.056383
56,PM10_CONC_7,0.048919
62,PM10_AQI_7,0.037785
198,PM10_AQI_24,0.030590
48,PM10_CONC_6,0.030136
110,PM10_AQI_13,0.026787
158,PM10_AQI_19,0.025535
54,PM10_AQI_6,0.025219


Among above top 50 features, PM10_DAY_AVE, PM2_5_DAY_AVE, SO2_DAY_AVE, NO2_DAY_AVE are from 'Beijing air quality data'! <p>
None of data from 'Fish data' was used ! 

#### model for after 3 days

In [16]:
tpot3 = TPOTRegressor(generations=5, population_size=50, verbosity=2, scoring='neg_mean_absolute_error', random_state=42)
tpot3.fit(X_train3, y_train3)
tpot3.export('tpot_seoul_PM10_AQI_3days.py')
print("neg MAE of TPOT Regressor : ",tpot3.score(X_test3, y_test3))


Generation 1 - Current best internal CV score: -12.370880795271477

Generation 2 - Current best internal CV score: -12.328013921282727

Generation 3 - Current best internal CV score: -12.229046275764976

Generation 4 - Current best internal CV score: -12.223733341820957

Generation 5 - Current best internal CV score: -12.223733341820957

Best pipeline: LinearSVR(LassoLarsCV(input_matrix, normalize=False), C=10.0, dual=False, epsilon=0.0001, loss=squared_epsilon_insensitive, tol=1e-05)
neg MAE of TPOT Regressor :  -11.87710435255352


->>> MAE is decreased to 15.46 from 11.88

### 4.2 Tpot Regressor light

#### model for tomorrow

In [4]:
tpotl = TPOTRegressor(generations=5, population_size=50, verbosity=2, scoring='neg_mean_absolute_error', random_state=42,config_dict="TPOT light")
tpotl.fit(X_train, y_train)
tpotl.export('tpot_light_seoul_PM10_AQI.py')
print("neg MAE of TPOT Regressor : ",tpotl.score(X_test, y_test))


Generation 1 - Current best internal CV score: -9.71911043158689

Generation 2 - Current best internal CV score: -9.641007047471618

Generation 3 - Current best internal CV score: -9.492929172851095

Generation 4 - Current best internal CV score: -9.492929172851095

Generation 5 - Current best internal CV score: -9.492929172851095

Best pipeline: LassoLarsCV(SelectFwe(input_matrix, alpha=0.007), normalize=True)
neg MAE of TPOT Regressor :  -9.19177756981145


->>> MAE is decreased to 9.19 from 10.63 (Tpot Regressor : 8.37)

* Features used for best pipeline (LassoLarsCV)

In [14]:
for i in range(0,len(features[0])):
    if tpotl.fitted_pipeline_.steps[-1][1].coef_[i]!=0 :
        print(sample.columns[i], "is used for prediction")

NO2_3 is used for prediction
PM10_AQI_4 is used for prediction
NO2_7 is used for prediction
PM10_AQI_12 is used for prediction
CO_13 is used for prediction
PM10_AQI_13 is used for prediction
PM10_CONC_19 is used for prediction
PM10_AQI_24 is used for prediction
PM10_DAY_AVE is used for prediction
PM2_5_DAY_AVE is used for prediction
SO2_DAY_AVE is used for prediction


Among above features, PM10_DAY_AVE, PM2_5_DAY_AVE, SO2_DAY_AVE are from 'Beijing air quality data'! <p>
None of data from 'Fish data' was used ! 

### 4.3 Tpot Regressor sparse

#### model for tomorrow

In [51]:
tpots = TPOTRegressor(generations=5, population_size=50, verbosity=2, scoring='neg_mean_absolute_error', random_state=42,config_dict="TPOT sparse")
tpots.fit(X_train, y_train)
tpots.export('tpot_sparse_seoul_PM10_AQI.py')
print("neg MAE of TPOT Regressor : ",tpots.score(X_test, y_test))


Generation 1 - Current best internal CV score: -9.505466770552744

Generation 2 - Current best internal CV score: -9.501146796890781

Generation 3 - Current best internal CV score: -9.448000868128494

Generation 4 - Current best internal CV score: -9.448000868128494

Generation 5 - Current best internal CV score: -9.42190925273874

Best pipeline: RandomForestRegressor(LinearSVR(RidgeCV(input_matrix), C=0.0001, dual=False, epsilon=0.001, loss=squared_epsilon_insensitive, tol=0.01), bootstrap=False, max_features=0.1, min_samples_leaf=8, min_samples_split=18, n_estimators=100)
neg MAE of TPOT Regressor :  -8.914622614897384


->>> MAE is decreased to 8.91 from 10.63


_____________________________

## Additional : Without Beijing Data 

In [ ]:
# drop the last row of the features
features_notB = sample.drop(['PM10_DAY_AVE','PM2_5_DAY_AVE','O3_DAY_AVE','NO2_DAY_AVE','CO_DAY_AVE','SO2_DAY_AVE'], axis=1).copy()
features_notB = features_notB .iloc[:-1, :].copy()
features3_notB = features_notB.iloc[:-2, :].copy()

features_notB = features_notB.to_numpy()
features3_notB = features3_notB.to_numpy()

# split into train and test set
X_train_notB, X_test_notB, y_train, y_test = train_test_split(features_notB.astype(np.float64), targets.astype(np.float64), train_size=0.8, test_size=0.2, shuffle=False)
X_train3_notB, X_test3_notB, y_train3, y_test3 = train_test_split(features3_notB.astype(np.float64), targets3.astype(np.float64), train_size=0.8, test_size=0.2, shuffle=False)

In [ ]:
tpotB = TPOTRegressor(generations=5, population_size=50, verbosity=2, scoring='neg_mean_absolute_error', random_state=42)
tpotB.fit(X_train_notB, y_train)
tpotB.export('tpot_seoul_PM10_AQI_withoutBeijing.py')
print("neg MAE of TPOT Regressor : ",tpot.score(X_test_notB, y_test))

In [ ]:
tpotB3 = TPOTRegressor(generations=5, population_size=50, verbosity=2, scoring='neg_mean_absolute_error', random_state=42)
tpotB3.fit(X_train3, y_train3)
tpotB3.export('tpot_seoul_PM10_AQI_3days_withoutBeijing.py')
print("neg MAE of TPOT Regressor : ",tpot.score(X_test3_notB, y_test3))